In [2]:
%load_ext autoreload
%autoreload 2
import sys
# map folder using sys
sys.path.append('../')

import vectorbt as vbt
from galgoz import DATA_FOLDER
from galgoz.utils import set_data_index_and_time_str
import pandas as pd
import numpy as np
import talib
from vectorbt import IndicatorFactory as IF
import datetime


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Load data from pkl file in DATA_FOLDER
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)
btc_price = vbt.YFData.download('BTC-USD',
                                missing_index='drop',
                                start=start_time,
                                end=end_time,
                                interval='1m').get("Close")
print(type(btc_price))
print(btc_price)
btc_price.info()

<class 'pandas.core.series.Series'>
Datetime
2025-02-06 09:37:00+00:00    98749.500000
2025-02-06 09:38:00+00:00    98708.812500
2025-02-06 09:39:00+00:00    98684.125000
2025-02-06 09:40:00+00:00    98671.812500
2025-02-06 09:41:00+00:00    98686.546875
                                 ...     
2025-02-08 09:28:00+00:00    96122.437500
2025-02-08 09:29:00+00:00    96178.664062
2025-02-08 09:30:00+00:00    96159.679688
2025-02-08 09:31:00+00:00    96163.875000
2025-02-08 09:32:00+00:00    96147.742188
Name: Close, Length: 2181, dtype: float64
<class 'pandas.core.series.Series'>
DatetimeIndex: 2181 entries, 2025-02-06 09:37:00+00:00 to 2025-02-08 09:32:00+00:00
Series name: Close
Non-Null Count  Dtype  
--------------  -----  
2181 non-null   float64
dtypes: float64(1)
memory usage: 34.1 KB


In [8]:
data = pd.read_pickle(DATA_FOLDER / "GBP_JPY_H1.pkl")
data = set_data_index_and_time_str(data)
print(type(data))
print(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
                            index  complete  volume    mid_o    mid_h  \
time                                                                    
2005-01-02 18:00:00+00:00       0         1       4  196.780  196.820   
2005-01-02 19:00:00+00:00       1         1      54  196.910  197.210   
2005-01-02 20:00:00+00:00       2         1     127  197.120  197.250   
2005-01-02 21:00:00+00:00       3         1     199  197.180  197.300   
2005-01-02 22:00:00+00:00       4         1     144  197.140  197.150   
...                           ...       ...     ...      ...      ...   
2025-02-06 19:00:00+00:00  127589         1    9454  188.390  188.436   
2025-02-06 20:00:00+00:00  127590         1    6476  188.266  188.520   
2025-02-06 21:00:00+00:00  127591         1    3760  188.498  188.508   
2025-02-06 22:00:00+00:00  127592         1    1132  188.372  188.414   
2025-02-06 23:00:00+00:00  127593         1    4544  188.308  188.318   

            

In [31]:
def custom_indicator(close, window=15):
    close_series = pd.Series(np.squeeze(close))
    sma = close_series.rolling(window=window).mean()
    return sma

# Create an indicator factory
ind = vbt.IndicatorFactory(
    class_name='CustomIndicator',
    short_name='custom',
    input_names=['close'], 
    param_names=['window'],
    output_names=['sma']).from_apply_func(custom_indicator, window=15)

res = ind.run(btc_price, window=21)
print(res.sma)


Datetime
2025-02-06 09:37:00+00:00             NaN
2025-02-06 09:38:00+00:00             NaN
2025-02-06 09:39:00+00:00             NaN
2025-02-06 09:40:00+00:00             NaN
2025-02-06 09:41:00+00:00             NaN
                                 ...     
2025-02-08 09:28:00+00:00    96040.070685
2025-02-08 09:29:00+00:00    96051.969866
2025-02-08 09:30:00+00:00    96063.433780
2025-02-08 09:31:00+00:00    96072.172247
2025-02-08 09:32:00+00:00    96079.684524
Name: (21, Close), Length: 2181, dtype: float64


In [35]:
close_series = pd.Series(np.squeeze(btc_price))
type(close_series)

pandas.core.series.Series

In [36]:
np.squeeze(btc_price)

Datetime
2025-02-06 09:37:00+00:00    98749.500000
2025-02-06 09:38:00+00:00    98708.812500
2025-02-06 09:39:00+00:00    98684.125000
2025-02-06 09:40:00+00:00    98671.812500
2025-02-06 09:41:00+00:00    98686.546875
                                 ...     
2025-02-08 09:28:00+00:00    96122.437500
2025-02-08 09:29:00+00:00    96178.664062
2025-02-08 09:30:00+00:00    96159.679688
2025-02-08 09:31:00+00:00    96163.875000
2025-02-08 09:32:00+00:00    96147.742188
Name: Close, Length: 2181, dtype: float64